In [4]:
#from fastapi import FastAPI, UploadFile, File, Form
import os
import json
import promptManager
import outputFormating
import pandas as pd


In [5]:

#Get marketing material
marketing_material = input("Enter marketing material: ") 

# Select product - as you add more products it would be better to start using IDs but we are using names for ease of reading here.
product=None
while product is None:
    product_num=input("1: Clareon® Toric Intraocular Lens (IOL) or 2:  PRECISION7® Contact Lenses")

    if product_num=="1":
        product="ClareonPanOptix"

    elif product_num=="2":
        product="TOTAL30"
    else:
        print("Invalid input, please enter 1 or 2.")
    
    print("Selected product: ", product)

Selected product:  TOTAL30


In [6]:
prompt=promptManager.buildPrompt(marketing_material, product)
print("Prompt built: ", prompt)

#Get response from LLM
output=promptManager.getResponseFromLLM(prompt)



Prompt built:  You are a compliance grader evaluating medical marketing content in the U.S.

Your goal is to:
1) Reward similarity with approved claims.
2) Penalize conflicts with FDA + FTC requirements.
3) Return structured JSON referencing policy IDs.

---

## Marketing Text
best eye product ever

---

## Policy Groups

Approved Claims (positive alignment)
- (approved:0) TOTAL30® contact lenses that feel like nothing, even at day 30.
- (approved:1) The first and only monthly replacement Water Gradient contact lenses.
- (approved:2) TOTAL30® contact lenses feature a gradual transition in water content, from 55 % at the core to nearly 100 % water at the outermost surface.
- (approved:3) Water Gradient Technology in TOTAL30 contact lenses lasts for a full 30 days.
- (approved:4) CELLIGENT® Technology creates a dynamic lens surface that biomimics the corneal surface.
- (approved:5) Helps resist the adherence of bacteria and lipids for a clean lens.
- (approved:6) Water Gradient delivers 

In [7]:
print("Output from LLM: ", output)

Output from LLM:  {'evaluations': [{'policy_id': 'approved:0', 'policy': 'TOTAL30® contact lenses that feel like nothing, even at day\u202f30.', 'type': 'approved', 'grade': 20, 'reason': "Text 'best eye product ever' does not specifically mention TOTAL30 or contact lenses feeling like nothing."}, {'policy_id': 'approved:1', 'policy': 'The first and only monthly replacement Water Gradient contact lenses.', 'type': 'approved', 'grade': 10, 'reason': 'No mention of monthly replacement or Water Gradient contact lenses.'}, {'policy_id': 'approved:2', 'policy': 'TOTAL30® contact lenses feature a gradual transition in water content, from 55\u202f% at the core to nearly 100\u202f% water at the outermost surface.', 'type': 'approved', 'grade': 10, 'reason': 'No technical detail or water content claim present.'}, {'policy_id': 'approved:3', 'policy': 'Water Gradient Technology in TOTAL30 contact lenses lasts for a full 30 days.', 'type': 'approved', 'grade': 10, 'reason': 'No mention of Water G

In [8]:

df = pd.DataFrame(output['evaluations'])


In [ ]:
df.head(60)

In [12]:
#get catagory scores
approved_scores = df[df['type'] == 'approved']['grade']
fda_scores = df[df['type'] == 'fda']['grade']
ftc_scores = df[df['type'] == 'ftc']['grade']

approved_score = approved_scores.max() if not approved_scores.empty else None

def compute_category_score(df, category, threshold=70):
    scores = df[df['type'] == category]['grade']

    if scores.empty:
        return 0
    
    base = scores.mean()
    non_compliant_ratio = (scores < threshold).mean()
    penalty = non_compliant_ratio * 100
    
    final = base - penalty
    return max(0, min(100, final))  # clamp

fda_score = compute_category_score(df, "fda")
ftc_score = compute_category_score(df, "ftc")

print("Approved Score: ", approved_score)
print("FDA Score: ", fda_score)
print("FTC Score: ", ftc_score)



Approved Score:  20
FDA Score:  90.0
FTC Score:  68.66666666666666
